# TensorBoard: Visualizing Learning
TensorFlowのプログラムの理解，デバッグ，最適化を容易にするための可視化ツールTensorFlowBoardについて学ぶ．  
計算グラフ，計算グラフの実行に対する定量評価結果などを可視化できる．  
このチュートリアルでは簡単な使い方を説明するだけので，詳細はTensorBoard READMEを参照のこと．  
[TensorBoard - github](https://github.com/tensorflow/tensorboard)

## Serializing the data (データをオブジェクトにまとめる)
TensorBoardはTensorFlowのイベントファイル(実行時に生成されるデータのまとまり)を読み込んで操作する．  
まずは，グラフを作り，どのノードのデータをまとめたいのか`summary operation`を使ってanotateする．  
例えば，学習係数がどう変化していったか，目的関数がどう変化していったかを見たければ，学習係数のノードや目的関数のノードに対して，`tf.summary.scalar`オペレーションを取り付ければよい．それぞれに`learning rate`や`loss function`のようにタグ付けしておく．タグ付けは`with tf.name_scope(learning_rate):`のようにスコープを作って行う．    
`tf.summary.histogram`のような別のオペレーションも用意されており，特定レイヤの発火状況や微分や重みの分布を見るのに使えたりする．  
詳細は[summary operations](https://www.tensorflow.org/api_guides/python/summary)を見ること．

TensorFlowのオペレーションはrunするまで何もしない．summaryノードもただ作っただけではだめ．summaryノードもrunする必要がある．  
手動でsummaryノードを管理するのはうんざり(tedious)なので，`tf.summary.merge_all`を使って一つのオペレーションにマージしてしまって，全てのsummaryデータを出力できるようにする．

マージした後は，そのsummaryオペレーションをrunすればよくて，そのオペレーションはrunされたステップ(?)時点でのまとまったデータ(a serialized Summary protobuf object)を生成する．  
最終的には，このsummaryデータを`tf.summary.FireWriter`に通すことでディスクに保存する．
(a serialized Summary protobuf object: 恐らくシリアライズフォーマットのProtocol Buffersの略としてprotobufと呼んでいる．)

FireWriterはコンストラクタ引数にログディレクトリGraphを受け取れる．
Graphを受け取った場合にはテンソルの形状情報に沿ってグラフを可視化できる．
[別のチュートリアル(TensorBoard: GraphVisualization)](https://www.tensorflow.org/get_started/graph_viz#tensor_shape_information)も見ると良い．

以下のコードは[MNIST For ML Beginners](https://www.tensorflow.org/get_started/mnist/beginners)の改変コード(抜粋)．
summaryオペレーションを幾つか加えており，10ステップごとにrunしている．(10ステップごとに記録している．)


全ソースコードは[mnist_with_summaries.py](https://github.com/tensorflow/tensorflow/blob/r1.3/tensorflow/examples/tutorials/mnist/mnist_with_summaries.py)．
この改変後のソースコードを実行すると，TensorBoardで可視化結果が見られる．

```python
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
  """Reusable code for making a simple neural net layer.

  It does a matrix multiply, bias add, and then uses relu to nonlinearize.
  It also sets up name scoping so that the resultant graph is easy to read,
  and adds a number of summary ops.
  """
  # Adding a name scope ensures logical grouping of the layers in the graph.
  with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
    with tf.name_scope('weights'):
      weights = weight_variable([input_dim, output_dim])
      variable_summaries(weights)
    with tf.name_scope('biases'):
      biases = bias_variable([output_dim])
      variable_summaries(biases)
    with tf.name_scope('Wx_plus_b'):
      preactivate = tf.matmul(input_tensor, weights) + biases
      tf.summary.histogram('pre_activations', preactivate)
    activations = act(preactivate, name='activation')
    tf.summary.histogram('activations', activations)
    return activations

hidden1 = nn_layer(x, 784, 500, 'layer1')

with tf.name_scope('dropout'):
  keep_prob = tf.placeholder(tf.float32)
  tf.summary.scalar('dropout_keep_probability', keep_prob)
  dropped = tf.nn.dropout(hidden1, keep_prob)

# Do not apply softmax activation yet, see below.
y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

with tf.name_scope('cross_entropy'):
  # The raw formulation of cross-entropy,
  #
  # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
  #                               reduction_indices=[1]))
  #
  # can be numerically unstable.
  #
  # So here we use tf.nn.softmax_cross_entropy_with_logits on the
  # raw outputs of the nn_layer above, and then average across
  # the batch.
  diff = tf.nn.softmax_cross_entropy_with_logits(targets=y_, logits=y)
  with tf.name_scope('total'):
    cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
  train_step = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(
      cross_entropy)

with tf.name_scope('accuracy'):
  with tf.name_scope('correct_prediction'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
  with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train',
                                      sess.graph)
test_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/test')
tf.global_variables_initializer().run()
```

In [1]:
# jupyterでダウンロードしてきた全ソースコードを実行
# Windows環境だと，引数を明示的に与えないと
# /tmp\tensorflow/mnist/logs/fully_connected_feedとなり、フォルダ生成に失敗する?(ュートリアルMechanics10と同じ．)
# jupyterで実行すると途中で落ちるので，別のコンソールで処理した
# run -i mnist_with_summaries.py --log_dir /tmp/tensorflow/mnist/logs/fully_connected_feed

## Launching TensorBoard (TensorBoardの起動)
TensorBoardを起動するには以下のコマンドを実行すれば良い．パスはとりあえず絶対パスで指定(?)
実行後は何も動かないように見えるが，`localhost:6006`にブラウザでアクセスすると結果が見られる．
```sh
tensorboard --logdir=c:/tmp/tensorflow/mnist/logs/fully_connected_feed
```